In [ ]:
import pandas as pd
import numpy as np
#----------------------------------------------------
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
import missingno as msno
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import PredictionErrorDisplay as PED
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
#----------------------------------------------------
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
#----------------------------------------------------


from flaml import AutoML

In [ ]:
ll=list(range(0,100001,1))
len(ll[0:100001:10])

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_new=pd.read_pickle("./calc_descriptors.pkl")
df_new

In [ ]:
def remove_correlated_features(descriptors):
    # Calculate correlation
    correlated_matrix = descriptors.corr().abs()

    # Upper triangle of correlation matrix
    upper_triangle = correlated_matrix.where(np.triu(np.ones(correlated_matrix.shape),k=1).astype(bool))

    # Identify columns that have above 0.9 values of correlation
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] >= 0.8)]
    print(to_drop)
    descriptors_correlated_dropped = descriptors.drop(columns=to_drop, axis=1)
    return descriptors_correlated_dropped

df_new = remove_correlated_features(df_new)
df_new

In [ ]:
df_new=df_new.dropna(axis='columns')
df_new.replace([np.inf, -np.inf], np.nan, inplace=True)
zero_cols = df_new.columns[(df_new == 0).all()]
df_new.drop(labels=zero_cols, axis=1, inplace=True)
df_new

In [ ]:
# credit to dataprofessor
# adapted from:https://github.com/dataprofessor/padel/blob/main/padelpy.ipynb

from sklearn.feature_selection import VarianceThreshold

def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

X = remove_low_variance(df_new, threshold=0.1)
X

In [ ]:
X[np.isfinite(X).any(axis=1)]

In [ ]:
X.to_pickle("./calc_descriptors_final.pkl")

In [ ]:
msno.bar(X)

In [ ]:
X.max(axis=0)

In [ ]:
X.min(axis=0)

In [ ]:
scaler=StandardScaler().fit(X)
scaled=scaler.transform(X)

In [ ]:
print(np.max(scaled),np.min(scaled))

In [ ]:
np.all(np.isfinite(scaled))

In [ ]:
df=pd.read_pickle("./gap_smile.pkl")

In [ ]:
y=df["GAP"][0:100001:10]
X_train, X_test, y_train, y_test = train_test_split(scaled,y,test_size=0.3,random_state=42)

In [ ]:
del df
del df_new


In [ ]:
automl = AutoML()
automl_settings = {
    "time_budget": 36000,
    "metric": 'r2',
    "task": 'regression',
    "log_file_name": 'gap.log',
#    "estimator_list": ['lgbm', 'xgboost', 'xgb_limitdepth', 'catboost', 'rf', 'extra_tree']
}
automl.fit(X_train = X_train, y_train = y_train, **automl_settings)

In [ ]:
y_pred=automl.predict(X_test)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
PED.from_predictions(
    y_test,
    y_pred=y_pred,
    kind="actual_vs_predicted",
    subsample=None,
    ax=axs[0],
    random_state=0,
)
axs[0].set_title("Actual vs. Predicted values")
axs[0].set_ylim(0,13)
axs[0].set_xlim(0,13)
axs[0].set_xticks(range(0,14,2))
axs[0].set_yticks(range(0,14,2))
PED.from_predictions(
    y_test,
    y_pred=y_pred,
    kind="residual_vs_predicted",
    subsample=None,
    ax=axs[1],
    random_state=0,
)
axs[1].set_title("Residuals vs. Predicted Values")
axs[1].set_xlim(0,13)
fig.suptitle("Plotting cross-validated predictions")
plt.tight_layout()
plt.show()
print(r2_score(y_pred=y_pred,y_true=y_test))

In [ ]:
automl.model

In [ ]:
automl.score

In [ ]:
automl.best_result

In [ ]:
automl.best_config

In [ ]:
X.columns

In [ ]:
f_imp=[[names,features] for names,features in zip(X.columns,automl.feature_importances_)]
#sns.displot(x=[n[1] for n in f_imp],legend=[n[0] for n in f_imp])
df_imp=pd.DataFrame(data=f_imp,columns=["features","importance"])
df_imp.sort_values(by="importance")

In [ ]:
600*(4*60+30)/24/60